In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import harp
import plotly.express as px

from harp_resources import process, utils
from sleap import load_and_process as lp

In [ ]:
cohort0 = False #only read harp data when it exists, not in Cohort0 
cohort2 = False

#Cohort 1 single OnixDigital file
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2717-2024-12-12T13-00-21') #single onix_digital files

#Cohort 1 multiple OnixDigital files 
data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2718-2024-12-12T13-28-14') #multiple onix_digital file

#Cohort 1 with clock accumulation issue marked on google sheet 
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2719-2024-12-12T13-59-38') #multiple onix_digital file

#Cohort 0 (no OnixHarp in this Cohort)
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort0/Cohort0_GCaMP_example/B3M3xx-2024-08-08T10-05-26')
#cohort0 = True

#Cohort 2 N.B. no videodata or photometry in this test set 
# cohort2 = True
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_like_test_data/2025-01-13T15-47-26')

#Cohort 2 longer test NO OnixHarp! Clock increasing exponentially according to NORA, but does not show uissue N.B. no photometry in this test set (neitjer videos, but yes video_data)
#cohort2 = True
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_test_longer/2025-02-07T16-05-04')

#Cohort 2 longer test YES OnixHarp! N.B. no photometry in this test set (neitjer videos, but yes video_data)
#cohort2 = True
#data_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort2_test_longer/2025-02-10T08-18-59')
 

photometry_path = data_path.parent / f"{data_path.name}_processedData" / "photometry"

# h1_datafolder = data_path / 'HarpDataH1' #only if reading separate registers
# h2_datafolder = data_path / 'HarpDataH2' #only if reading separate registers

In [ ]:
#h1 and h2 only needed if timestamps are readed separately and not as all harp_streams
# h1_reader = harp.create_reader('harp_resources/h1-device.yml', epoch=harp.REFERENCE_EPOCH)
# h2_reader = harp.create_reader('harp_resources/h2-device.yml', epoch=harp.REFERENCE_EPOCH)

session_settings_reader = utils.SessionData("SessionSettings")
experiment_events_reader = utils.TimestampedCsvReader("ExperimentEvents", columns=["Event"])
onix_framecount_reader = utils.TimestampedCsvReader("OnixAnalogFrameCount", columns=["Index"])
#photometry_reader = utils.PhotometryReader("Processed_fluorescence")
video_reader1 = utils.Video("VideoData1")
video_reader2 = utils.Video("VideoData2")
onix_digital_reader = utils.OnixDigitalReader("OnixDigital", columns=["Value.Clock", "Value.HubClock", 
                                                                         "Value.DigitalInputs",
                                                                         "Seconds"])
onix_harp_reader = utils.TimestampedCsvReader("OnixHarp", columns=["Clock", "HubClock", "HarpTime"])

In [ ]:
#read metadata in 2 different ways (to df or to dict, to decide which one is better in the future)
print ("Loading session settings")
session_settings = utils.load_2(session_settings_reader, data_path) #Andrew's, creates ugly df, but used in further analysis code
#session_settings = utils.read_SessionSettings(data_path) #Hilde's, creates prety dict, not aware of multiple files

# read experiment events, video, processed photometry 
print ("Loading experiment events")
experiment_events = utils.load_2(experiment_events_reader, data_path)
if not cohort2:
    print ("Loading processed fluorescence")
    photometry_data=pd.read_csv(str(photometry_path)+'/Processed_fluorescence.csv')
    print ("Loading processed fluorescence info")
    photometry_info=pd.read_csv(str(photometry_path)+'/Info.csv')
    print ("Loading processed fluorescence events")
    photometry_events=pd.read_csv(str(photometry_path)+'/Events.csv')
    print ("Loading video data 1")
    video_data1 = utils.load_2(video_reader1, data_path)
    print ("Loading video data 2")
    video_data2 = utils.load_2(video_reader2, data_path)

# read Onix data 
print ("Loading OnixDigital")
onix_digital = utils.load_2(onix_digital_reader, data_path)
print ("Loading OnixAnalogFrameClock")
onix_analog_framecount = utils.load_2(onix_framecount_reader, data_path)
print ("Loading OnixAnalogClock")
onix_analog_clock = utils.read_OnixAnalogClock(data_path)
print ("Loading OnixAnalogData")
onix_analog_data = utils.read_OnixAnalogData(data_path, channels = [0], binarise=True, method = 'adaptive', verbose = True) #channels is a list of AI lines, 0-11
halt_count = (experiment_events["Event"] == "Halt").sum()
print(f'Halts in exp_events: {halt_count}, same as falling edges?')

#read harp streams and separate registers if needed 
print ("Loading H1 and H2 streams as dict or df")
harp_streams = utils.load_registers(data_path, dataframe = True) #loads as df, or if False, as dict 

#read syncronising signal between HARP and ONIX
if not cohort0:
    print ("Loading OnixHarp")
    onix_harp = utils.load_2(onix_harp_reader, data_path)
    # removing possible outliers 
    onix_harp = utils.detect_and_remove_outliers(
    df=onix_harp,
    x_column="HarpTime",
    y_column="Clock",
    verbose=False  # True prints all outliers
    )

# print (" ")
# print ("loading separate registers from H1 and H2 data")
# print ("Loading camera triggers")
# camera_triggers = utils.load_harp(h1_reader.Cam0Event, h1_datafolder) #assumes Cam0 triggers both cameras
# print ("Loading flow sensor data")
# flow_sensor = utils.load_harp(h1_reader.OpticalTrackingRead, h1_datafolder)
print ("Done Loading")

In [ ]:
import importlib
importlib.reload(utils)
importlib.reload(process) # Forces Python to reload the updated module
None
onix_analog_data = utils.read_OnixAnalogData(data_path, channels = [0], binarise=True, method = 'adaptive', refractory = 300, verbose = True) #channels is a list of AI lines, 0-11


In [ ]:
import plotly.graph_objects as go

def plot_data(start_time_sec=0, end_time_sec=1):
    """
    Plots the specified time window of harp_streams and experiment_events.

    Parameters:
    - start_time_sec (int): Start of the time window (in seconds).
    - end_time_sec (int): End of the time window (in seconds).
    """
    
    # Define downsampling factors
    harp_streams_downsample_factor = 10
    onix_analog_data_downsample_factor = 10000
    experiment_events_downsample_factor = 1  

    # ✅ Create a copy of harp_streams before modification
    harp_streams_copy = harp_streams.copy()

    # Interpolate and fix NaNs in OpticalTrackingRead0Y(46) in the copied dataframe
    harp_streams_copy["OpticalTrackingRead0Y(46)"] = (
        harp_streams_copy["OpticalTrackingRead0Y(46)"]
        .interpolate(method="linear", limit_direction="both")
    )

    # Convert datetime index to elapsed seconds
    start_time = harp_streams_copy.index[0]  
    harp_streams_copy["ElapsedTime"] = (harp_streams_copy.index - start_time).total_seconds()
    experiment_events["ElapsedTime"] = (experiment_events.index - start_time).total_seconds()

    # ✅ Filter data based on the time window
    harp_streams_filtered = harp_streams_copy[
        (harp_streams_copy["ElapsedTime"] >= start_time_sec) &
        (harp_streams_copy["ElapsedTime"] <= end_time_sec)
    ]
    
    experiment_events_filtered = experiment_events[
        (experiment_events["ElapsedTime"] >= start_time_sec) &
        (experiment_events["ElapsedTime"] <= end_time_sec)
    ]

    # Downsample data
    harp_streams_downsampled = harp_streams_filtered.iloc[::harp_streams_downsample_factor]
    onix_analog_data_1d = onix_analog_data.squeeze()
    onix_analog_data_downsampled = onix_analog_data_1d[:len(harp_streams_filtered)][::onix_analog_data_downsample_factor]
    experiment_events_downsampled = experiment_events_filtered.iloc[::experiment_events_downsample_factor]

    # ✅ Plot Optical Tracking (Now fully interpolated, but original `harp_streams` is unchanged)
    fig1 = go.Figure()

    fig1.add_trace(go.Scatter(
        x=harp_streams_downsampled.index,
        y=harp_streams_downsampled["OpticalTrackingRead0Y(46)"],
        mode="lines",
        name="Optical Tracking Read Y"
    ))

    fig1.update_layout(
        title=f"Optical Tracking Read Y ({start_time_sec}-{end_time_sec} sec)",
        xaxis_title="Time",
        yaxis_title="Values",
        dragmode="pan"
    )

    fig1.show()

    # ✅ Plot Analog Data
    fig2 = go.Figure()

    fig2.add_trace(go.Scatter(
        x=harp_streams_downsampled.index[:len(onix_analog_data_downsampled)],
        y=onix_analog_data_downsampled,
        mode="lines",
        name="Analog Data"
    ))

    fig2.update_layout(
        title=f"Analog Data ({start_time_sec}-{end_time_sec} sec)",
        xaxis_title="Time",
        yaxis_title="Values",
        dragmode="pan"
    )

    fig2.show()

    # ✅ Plot Experiment Events (Scatter)
    fig3 = go.Figure()

    fig3.add_trace(go.Scatter(
        x=experiment_events_downsampled.index,
        y=experiment_events_downsampled["Event"],
        mode="markers",
        marker=dict(color="black"),
        name="Experiment Events"
    ))

    fig3.update_layout(
        title=f"Experiment Events Over Time ({start_time_sec}-{end_time_sec} sec)",
        xaxis_title="Time",
        yaxis_title="Event",
        dragmode="pan"
    )

    fig3.show()


In [ ]:
#plot_data(start_time_sec=0, end_time_sec=3600)

issue is that analog data does not have timestamp... need to do the conversion first for analog_data, then for harp with onix_harp